In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = f'''https://www.auveco.com/api/cacheable/items?commercecategoryurl={input_.loc[a, 'Url'].split('://www.auveco.com')[1].split('?')[0]}&country=US&currency=USD&fieldset=search&include=facets&language=en&pricelevel=1&sort=commercecategory%3Adesc&use_pcv=T'''

            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(f'''{url}&limit=1&offset=0''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        json_ = resp.json()
                        
                        if json_['code'] == 200:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if json_['total'] <= 0:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': url,
                                     'SKU Count': json_['total'],
                                     'Page': math.ceil(json_['total'] / 100)}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/1.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/page_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[状态：ok，尝试次数：4] - 2.https://www.auveco.com/products/bodyhardware/boltsbodyhardware/bodybolts
[剩余数量：0] - [当前时间：14:53:13]

[状态：ok，尝试次数：27] - 1.https://www.auveco.com/products/assortments/cabinetandcompartmentbinassortments
[剩余数量：0] - [当前时间：14:53:14]

输出ing...

Done ~
